In [ ]:

import numpy as np
import os
from scipy.integrate import solve_ivp
from numba import njit, prange
import time


In [ ]:

@njit(parallel=True)
def fast_interaction_loop(f_local, nv, K, K_img, imgvortex_vectx, imgvortex_vecty,
                          omega_c, phi, V_0, Xi, a, R, nmax):
    dfdt = np.zeros(2 * nv, dtype=np.float64)

    for i in prange(nv):
        xi = f_local[i]
        yi = f_local[i + nv]
        Ki = K[i]
        if Ki == 0:
            continue
        dxdt = omega_c * yi
        dydt = -omega_c * xi

        for j in range(nv):
            if i == j or K[j] == 0:
                continue
            xj = f_local[j]
            yj = f_local[j + nv]
            dx = xi - xj
            dy = yi - yj
            r2 = dx * dx + dy * dy
            dxdt += -K[j] * Ki * dy / r2
            dydt += K[j] * Ki * dx / r2

            dx_img = xi - imgvortex_vectx[j]
            dy_img = yi - imgvortex_vecty[j]
            r2_img = dx_img * dx_img + dy_img * dy_img
            dxdt += K_img[j] * Ki * dy_img / r2_img
            dydt += -K_img[j] * Ki * dx_img / r2_img

        nx = int(round((xi + R) / a))
        ny = int(round((yi + R) / a))
        if nx <= 0 or ny <= 0 or nx >= nmax or ny >= nmax:
            nx, ny = 0, 0

        x_pin = -R + round((xi + R) / a) * a
        y_pin = -R + round((yi + R) / a) * a
        r2_pin = (x_pin - xi)**2 + (y_pin - yi)**2
        force_factor = V_0[nx, ny] * np.exp(-r2_pin / (2 * Xi**2))
        dxdt += force_factor * (yi - y_pin)
        dydt += -force_factor * (xi - x_pin)

        vel_x = dxdt * Ki
        vel_y = dydt * Ki
        dfdt[i] = np.cos(phi) * vel_x + np.sin(phi) * vel_y
        dfdt[i + nv] = -np.sin(phi) * vel_x + np.cos(phi) * vel_y

    return dfdt


In [ ]:

def eom_wrapper(t, f_local):
    return fast_interaction_loop(
        np.asarray(f_local, dtype=np.float64),
        nv,
        np.asarray(K, dtype=np.float64),
        np.asarray(K_img, dtype=np.float64),
        np.asarray(imgvortex_vectx, dtype=np.float64),
        np.asarray(imgvortex_vecty, dtype=np.float64),
        omega_c,
        phi,
        np.asarray(V_0, dtype=np.float64),
        Xi,
        a,
        R,
        nmax
    )


In [ ]:

def save_checkpoint(step_id, prefix="checkpoint"):
    np.save(os.path.join(pathplace, f"{prefix}_f.npy"), np.asarray(f, dtype=np.float32))
    np.save(os.path.join(pathplace, f"{prefix}_omega_c_vec.npy"), np.asarray(omega_c_vec, dtype=np.float32))
    np.save(os.path.join(pathplace, f"{prefix}_omega_s_vec.npy"), np.asarray(omega_s_vec, dtype=np.float32))
    np.save(os.path.join(pathplace, f"{prefix}_tvalues.npy"), np.asarray(tvalues, dtype=np.float32))

def load_checkpoint(prefix="checkpoint"):
    global f, omega_c_vec, omega_s_vec, tvalues
    f = np.load(os.path.join(pathplace, f"{prefix}_f.npy")).tolist()
    omega_c_vec = np.load(os.path.join(pathplace, f"{prefix}_omega_c_vec.npy")).tolist()
    omega_s_vec = np.load(os.path.join(pathplace, f"{prefix}_omega_s_vec.npy")).tolist()
    tvalues = np.load(os.path.join(pathplace, f"{prefix}_tvalues.npy")).tolist()


In [ ]:

def simulation_with_checkpointing():
    global t, omega_c
    last_saved_percent = 0

    while t <= runtime_simulation:
        progress_percent = int((t / runtime_simulation) * 100)

        if progress_percent >= last_saved_percent + 5:
            print(f"[Checkpoint] Saving simulation at {progress_percent}%")
            save_checkpoint(step_id=progress_percent, prefix="sim")
            last_saved_percent = progress_percent

        write_progress(runtime_simulation, "Simulation")
        tvalues.append(t / T_0)
        vortex_states.append(list(f))
        omega_c_vec.append(omega_c / omega_0)
        omega_s_vec.append(omega_s / omega_0)
        unpinned_vec.append(unpinned_count)
        vortex_in_vec.append(vortex_in_count)

        if trig_duration != 0:
            trigger_check(duration_trig, t)

        omega_c += N_ext * del_t
        integ_adaptive()
        t += del_t

        updates(omega_s_condition=1)

        if len(omega_s_vector) >= 2:
            delta_omega_s = omega_s_vector[-1] - omega_s_vector[-2]
            omega_c -= delta_omega_s


In [ ]:

def resume_simulation_from_checkpoint():
    global t, last_saved_percent
    load_checkpoint(prefix="sim")
    t = tvalues[-1] * T_0 if tvalues else 0
    last_saved_percent = int((t / runtime_simulation) * 100)
    print(f"Resuming simulation from t = {t:.2f}, which is {last_saved_percent}% complete")
    simulation_with_checkpointing()
